In [ ]:
import tensorflow as tf
from GAN import GAN
import numpy as np
import os, copy
import matplotlib.pyplot as plt
from Vgg19 import Vgg19
from imagenet_classes import class_names
import pandas as pd
%matplotlib inline

In [ ]:
all_birds = pd.read_csv('NACC_list_species.csv', index_col=0)[["common_name", "species"]]
all_birds.common_name, all_birds.species = all_birds.common_name.str.lower(), all_birds.species.str.lower()

class_names_mod=[]

for c in class_names:
    for sp in c.split(","):
        if sp in list(all_birds.common_name) or sp in list(all_birds.species):
            class_names_mod.append(1)
        else:
            class_names_mod.append(0)
        break;
    

In [ ]:
z_dim = 1024
tf.reset_default_graph()
gan = GAN(verbose=False)
layers = list()


G_layers = [[1024, [4, 4], (1, 1), "valid", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [512, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [256, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [128, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [64, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [32, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [3, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 0, "th", None, 0]]

graph, sess_gan, z, G_col, G_out = gan.build_inference_graph(G_layers, tf.placeholder(tf.float32, [None, 1, 1, z_dim]), 
                                                             "lsun_generator_vars.npz")
                                                         #"Model\\GAN_RMS_256_with_noise_mini_sm_an_z-199999")
    
z_val = np.random.normal(0, 1.0, size=[100, 1, 1, 1024])
img = sess_gan.run(tf.clip_by_value(G_out, -1, 1), feed_dict={z:z_val})/2 + 0.5  

graph_vgg = tf.Graph()
with graph_vgg.as_default():
    vgg19_m = Vgg19()
    vgg_img = tf.placeholder(shape=[100, 224, 224, 3], dtype=tf.float32)
    vgg19_m.build(vgg_img)

with tf.Session(graph=graph_vgg) as sess:
    img_resized = sess.run(tf.image.resize_images(img, (224, 224)))
    feed_dict = {vgg_img:img_resized}
    probs, cat = sess.run([vgg19_m.prob, tf.argmax(vgg19_m.prob, axis=1)], feed_dict=feed_dict)
    

cat_detected = z_val[[i for i, d in enumerate(cat) if class_names_mod[d]==1], ...]
img_birds = sess_gan.run(tf.clip_by_value(G_out, -1, 1), feed_dict={z:cat_detected})/2 + 0.5  
sess_gan.close()
nrows, ncols = img_birds.shape[0]//4+img_birds.shape[0]%4,  4
f, ax =plt.subplots(nrows=nrows, ncols=ncols)
f.set_size_inches(20, 20)
#z_codes=list()
for i, (r, c) in enumerate([(r, c) for r in np.arange(nrows) for c in np.arange(ncols)]):
    if i>img_birds.shape[0]-1:
        break;
    ax[r][c].set_title(i)
    #ax[r][c].imshow(sess.run(tf.clip_by_value(G_out, -1, 1), feed_dict={z:cat}).squeeze()/2 + 0.5)
    ax[r][c].imshow(img_birds[i,...])

plt.show()

In [ ]:
good_ones = np.load("good_ones.npy")
good_ones = np.concatenate([good_ones, cat_detected[[4, 5, 6, 9], ...]], axis=0)
np.save("good_ones.npy", good_ones)